In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import spacy
import sys
sys.path.append('../../../../utils')
sys.path.append('..')
import gezi
from gezi import tqdm
from src.eval import calc_f1
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [2]:
import tensorflow as tf
from absl import flags
FLAGS = flags.FLAGS
from transformers import AutoTokenizer, AutoModel
from src import config
from src.model import Model
from src.dataset import Dataset
from src.util import get_preds
import src.eval as ev
import melt as mt
import husky

In [3]:
import torch
from torch import nn
from torch.nn import functional as F
import lele

In [4]:
class Model(nn.Module):
  
  def __init__(self, **kwargs):
    super().__init__(**kwargs)  
    self.backbone = AutoModel.from_pretrained(FLAGS.backbone)
    
    dim = self.backbone.config.hidden_size
    self.dense = nn.Linear(dim, FLAGS.num_classes)
    lele.keras_init(self, True, True)
    
    self.eval_keys = ['id', 'label', 'mask', 'word_ids']
    self.str_keys = ['id']
    self.out_keys = []

  def forward(self, inputs):
    x = self.backbone(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])[0]
    x = self.dense(x)
    return x
  
  def get_loss_fn(self):
    def calc_loss(y_, y):
      y_ = y_.view(-1, FLAGS.num_classes)
      y = y.view(-1)
      return nn.CrossEntropyLoss()(y_, y)
    return calc_loss


In [5]:
FLAGS([''])
FLAGS.mts = True
FLAGS.torch = True
FLAGS.optimizer = 'bert-Adam'
config.init_()
config.init()
mt.init()

ic| config.py:57 in init_()
    FLAGS.backbone: '../input/allenai/longformer-base-4096'
ic| config.py:63 in init_()- FLAGS.num_classes: 15
ic| config.py:57 in init_()
    FLAGS.backbone: '../input/allenai/longformer-base-4096'
ic| config.py:63 in init_()- FLAGS.num_classes: 15
ic| config.py:72 in init()
    FLAGS.valid_files[:2]: ['../input/feedback-prize-2021/tfrecords/train/70.164.tfrec',
                            '../input/feedback-prize-2021/tfrecords/train/90.164.tfrec']
ic| init.py:200 in init()
    tf.__version__: '2.6.2'
    torch.__version__: '1.9.0'
ic| init.py:334 in init()
    FLAGS.model_dir: '../working/offline/0/base.20211229-052906'
    FLAGS.log_dir: '../working/offline/0/base.20211229-052906'
2021-12-29 05:29:07 0:00:00 batch size is shrink by 4 for each gpu to make total insts per step still 32
2021-12-29 05:29:07 0:00:00 fcntl.floc with lock_file /home/huigecheng/.melt.lock (If hang here means other programs calling melt.init have not finished yet)
2021-12-29 05:2

In [6]:
fit = mt.fit
strategy = mt.distributed.get_strategy()
with strategy.scope():    
  model = Model()

  fit(model,  
    Dataset=Dataset,
    loss_fn=model.get_loss_fn(),
    eval_fn=ev.evaluate
    ) 

Some weights of the model checkpoint at ../input/allenai/longformer-base-4096 were not used when initializing LongformerModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerModel were not initialized from the model checkpoint at ../input/allenai/longformer-base-4096 and are newly initialized: ['longformer.pooler.dense.weight', 'longformer.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2021-12-29 05:30:04 0:00:57 Round: 0 mode:

get_num_records:   0%|          | 0/76 [00:00<?, ?it/s]

ic| train.py:1194 in train()
    len(valid_inputs): 19
    valid_inputs[:2]: ['../input/feedback-prize-2021/tfrecords/train/0.165.tfrec',
                       '../input/feedback-prize-2021/tfrecords/train/5.165.tfrec']
2021-12-29 05:30:13 0:01:05 num_train_examples: 12475 num_steps_per_epoch: 390


get_num_records:   0%|          | 0/19 [00:00<?, ?it/s]

2021-12-29 05:30:13 0:01:05 num_valid_examples: 3119 num_valid_steps_per_epoch: 98
2021-12-29 05:30:13 0:01:06 ----------------Model dir: ../working/offline/0/base.20211229-052906


...

None-None-eval:   0%|          | 0/98 [00:00<?, ?it/s]

  0%|          | 0/3119 [00:00<?, ?it/s]

2021-12-29 05:37:37 0:08:29 WARNING: eval fn error
2021-12-29 05:37:37 0:08:29 WARNING: Traceback (most recent call last):
  File "/home/huigecheng/.local/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2898, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1675, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1683, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'overlaps'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/huigecheng/.local/lib/python3.6/site-packages/pandas/core/generic.py", line 3576, in _set_item
    loc = self._info_axis.get_loc(key)
  File "/home/huigecheng/.local/lib/python3.6/sit

Epoch: 3/3:   0%|          | 0/390 [00:00<?, ?it/s]

None-None-eval:   0%|          | 0/98 [00:00<?, ?it/s]